<a href="https://colab.research.google.com/github/pouyarahmti/Pytorch-Deep-Learning-Bootcamp/blob/main/08_pytorch_paper_replicating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pytorch Paper Replicaing

The goal of machine learning paper replicating is: turn a ML research paper into usable code.

In this notebook we are going to replicate Vision Transformer (ViT) architecture/paper with PyTorch: https://arxiv.org/abs/2010.11929

In [2]:
import torch
import torchvision


print(f"PyTorch version: {torch.__version__}")
print(f"Torchvision version: {torchvision.__version__}")

PyTorch version: 2.5.1+cu121
Torchvision version: 0.20.1+cu121
